In [91]:
import bs4
import requests
import re
import time
import random as ran
import sys
import pandas as pd

In [92]:
url = 'https://loigiaihay.com/trac-nghiem-toan-8-bai-26-ket-noi-tri-thuc-co-dap-an-a153391.html'

source = requests.get(url).text
soup = bs4.BeautifulSoup(source,'html.parser')

In [93]:
title = soup.title.text;title

'Trắc nghiệm toán 8 bài 26 kết nối tri thức có đáp án'

In [94]:
grade = 8

In [95]:
# so long, just for experiment
# soup.find_all('div',class_='question')

[<div class="question c64a29f81efb726e3b3849384">
 <div class="question-content">
 <div class="question-content-inner">
 <strong class="question-index-number">Câu 1 : </strong>
 <p>Bạn Nga dành mỗi ngày x phút để chạy bộ. Biểu thức biểu thị quãng đường (đơn vị: m) bạn Nga chạy được trong x phút với vận tốc 160m/phút là: </p> </div>
 </div>
 <ul class="dapan rows-4">
 <li style="display: flex;align-items:flex-start">
 <span class="span-answer" style="padding-top: 0;">A.</span>
 <div style="width: 90%">
                         \(160 - x\left( m \right)\)                    </div>
 </li>
 <li style="display: flex;align-items:flex-start">
 <span class="span-answer" style="padding-top: 0;">B.</span>
 <div style="width: 90%">
                         \(160 + x\left( m \right)\)                    </div>
 </li>
 <li style="display: flex;align-items:flex-start">
 <span class="span-answer" style="padding-top: 0;">C.</span>
 <div style="width: 90%">
                         \(160x\left( m \righ

## start from here

In [96]:
import re
def preprocess_text(text):
    # Remove HTML tags
    text = re.sub(r'<[^>]+>', '', text)
    # Remove extra spaces
    text = re.sub(r'\s+', ' ', text)
    # Tokenize the text
    tokens = text.split()
    # Lowercase all tokens
    tokens = [token.lower() for token in tokens]
    # Remove stopwords (if needed)
    # Lemmatize tokens (if needed)
    # Join tokens back into text
    preprocessed_text = ' '.join(tokens)
    return preprocessed_text
    return preprocessed_text
for tag in soup.find_all('div',class_='question')[:1]:
    print(preprocess_text(tag.text))
    

câu 1 : bạn nga dành mỗi ngày x phút để chạy bộ. biểu thức biểu thị quãng đường (đơn vị: m) bạn nga chạy được trong x phút với vận tốc 160m/phút là: a. \(160 - x\left( m \right)\) b. \(160 + x\left( m \right)\) c. \(160x\left( m \right)\) d. \(\frac{x}{{160}}\left( m \right)\)


**It looks good. Its time to use all**

In [97]:
from dotenv import dotenv_values
hf_token = dotenv_values("../.env")['HF_TOKEN']
from huggingface_hub import login
login(token=hf_token)

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.


Token is valid (permission: write).
Your token has been saved to /home/pc/.cache/huggingface/token
Login successful


In [98]:
from datasets import load_dataset, Dataset

In [99]:
problems = [preprocess_text(tag.text) for tag in soup.find_all('div',class_='question')[:]]
# problems

## time to push

In [100]:
dataset_name = 'loigiaihay_toan8'

In [101]:
my_ds = Dataset.from_dict({'grade':[grade]*len(problems), 
                           "title" :[title]*len(problems),
                           'problem':problems, 
                           "url":[url]*len(problems)})

In [102]:
my_ds

Dataset({
    features: ['grade', 'title', 'problem', 'url'],
    num_rows: 60
})

In [103]:
# upload if needed
# my_ds.push_to_hub(dataset_name)